In [2]:
#! pip install "numpy<1.17"

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train.shape

Using TensorFlow backend.


(60000, 28, 28)

In [4]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    def __init__(self,units=128, input_dim=128):
        super(Linear,self).__init__()
        w_init=tf.contrib.layers.xavier_initializer()
        self.weights2=tf.Variable(initial_value=w_init(shape=(input_dim,units),
                                                      dtype=tf.float32),trainable=True)
        b_init=tf.contrib.layers.xavier_initializer()
        self.bias2=tf.Variable(initial_value=b_init(shape=(units,),dtype=tf.float32),
                              trainable=True)
    def call(self,inputs):
        return tf.matmul(inputs,self.weights2)+self.bias2

In [31]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

batch_size=5000

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    #tf.set_random_seed(2)
    
    X = tf.placeholder(dtype=tf.float32)
    #X2 = tf.placeholder(dtype=tf.float32)
    Y = tf.placeholder(dtype=tf.float32)

    num_hidden=128
    
    linear_layer = Linear(batch_size,num_hidden)


    # Build a hidden layer
    #W_hidden = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[784, num_hidden]))
    #b_hidden = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden]))
    #p_hidden = tf.nn.relu( tf.add(tf.matmul(X, W_hidden), b_hidden) )

    W_hidden2 = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[784, num_hidden]))
    b_hidden2 = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden]))
    p_hidden22 = tf.add(tf.matmul(X, W_hidden2), b_hidden2)
    p_hidden3 = tf.layers.batch_normalization(tf.reshape(p_hidden22,[batch_size,num_hidden]))
    p_hidden22 = tf.nn.relu(p_hidden3)

    # 
    W_concat = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[num_hidden,num_hidden]))
    b_concat = tf.Variable(tf.zeros(shape=[num_hidden]))
    p_concat2 = tf.add(tf.matmul(p_hidden22, W_concat), b_concat)
    p_concat3 = tf.layers.batch_normalization(tf.reshape(p_concat2,[batch_size,num_hidden]))
    linear1=linear_layer(p_concat3)
    p_concat22=tf.nn.relu(linear1)
    
    # Build the output layer
    W_output = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[batch_size, 10]))
    b_output = tf.Variable(tf.contrib.layers.xavier_initializer()(shape=[10]))
    p_output = tf.nn.softmax( tf.add(tf.matmul(p_concat22, W_output), b_output))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=p_output))
    acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(Y,1), 
                                      predictions=tf.argmax(p_output,1))
    
    minimization_op = tf.train.GradientDescentOptimizer(learning_rate=0.06).minimize(loss)
    saver = tf.train.Saver()
    
    limit=10000
    
    def norm(x):
        return (x-np.min(x))/(np.max(x)-np.min(x))

    feed_dict = {
        X: norm(x_train.reshape(-1,784)),
        #X2: norm(x_train.reshape(-1,784)),
        Y: pd.get_dummies(y_train)
    }

/home/rubensvectomobile_gmail_com/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [32]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle).astype(np.float32), np.asarray(labels_shuffle).astype(np.float32)

In [34]:
with tf.Session(graph=graph, config=config) as session:
    ckpt = "/home/rubensvectomobile/Scratch/Train/train7/model.ckpt"
    saver.restore(session, ckpt)

    #init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    #session.run(init)
    
    for step in range(1300):

        X0, Y0 = next_batch(batch_size, x_train, y_train)
        X02=X0
        
        feed_dict = {
        X: norm(X0.reshape(-1,784)),
        #X2: norm(X02.reshape(-1,784)),
        Y: pd.get_dummies(Y0)}

        J_value = session.run(loss, feed_dict)
        acc = (batch_size-session.run(tf.count_nonzero(tf.add(tf.argmax(session.run(Y, feed_dict),1),-tf.argmax(session.run(p_output, feed_dict),1)))))/batch_size

        session.run(minimization_op, feed_dict)
        if step % 10 == 0:
            print("Step:", step, " Loss:", J_value," Accuracy:", acc)
            if acc > 0.99:
                save_path = saver.save(session, "/home/rubensvectomobile/Scratch/Train/train1/model.ckpt")
                print("Model saved in path: %s" % save_path)

Step: 0  Loss: 1.4762019  Accuracy: 0.9866
Step: 10  Loss: 1.4756101  Accuracy: 0.9872
Step: 20  Loss: 1.475983  Accuracy: 0.9868
Step: 30  Loss: 1.4786718  Accuracy: 0.9838
Step: 40  Loss: 1.4771191  Accuracy: 0.986
Step: 50  Loss: 1.4770303  Accuracy: 0.9852
Step: 60  Loss: 1.475833  Accuracy: 0.9872
Step: 70  Loss: 1.4763572  Accuracy: 0.9858
Step: 80  Loss: 1.476984  Accuracy: 0.9854
Step: 90  Loss: 1.4757655  Accuracy: 0.9868
Step: 100  Loss: 1.4775071  Accuracy: 0.985
Step: 110  Loss: 1.4815911  Accuracy: 0.9808
Step: 120  Loss: 1.4787176  Accuracy: 0.9836
Step: 130  Loss: 1.475704  Accuracy: 0.987
Step: 140  Loss: 1.4749098  Accuracy: 0.988
Step: 150  Loss: 1.4775015  Accuracy: 0.9852
Step: 160  Loss: 1.480595  Accuracy: 0.982
Step: 170  Loss: 1.4776415  Accuracy: 0.9852
Step: 180  Loss: 1.4779752  Accuracy: 0.9844
Step: 190  Loss: 1.4756364  Accuracy: 0.9868
Step: 200  Loss: 1.4779478  Accuracy: 0.9848
Step: 210  Loss: 1.4824313  Accuracy: 0.98
Step: 220  Loss: 1.4795115  Accur

In [37]:
with tf.Session(graph=graph, config=config) as session:
    ckpt = "/home/rubensvectomobile/Scratch/Train/train7/model.ckpt"
    saver.restore(session, ckpt)

    X02, Y02 = next_batch(5000, x_test, y_test)

    feed_dict = {
    X: norm(X02.reshape(-1,784)),
    #X2: norm(X02.reshape(-1,784)),
    Y: pd.get_dummies(Y0)}

    prev = session.run(p_output, feed_dict)

In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y02, np.argmax(prev,axis=1).reshape(5000,-1))

array([[484,   0,   0,   1,   0,   1,   2,   1,   0,   2],
       [  0, 548,   1,   0,   0,   0,   1,   1,   1,   0],
       [  2,   0, 490,   4,   1,   1,   2,   5,   2,   0],
       [  0,   0,   2, 510,   0,   4,   0,   4,   3,   1],
       [  1,   0,   1,   0, 450,   1,   4,   2,   0,   8],
       [  2,   0,   0,   5,   0, 438,   3,   0,   3,   1],
       [  6,   3,   0,   0,   2,   5, 484,   0,   0,   0],
       [  0,   4,   8,   1,   1,   0,   0, 518,   1,   1],
       [  2,   0,   1,   5,   3,   4,   2,   4, 456,   4],
       [  0,   3,   0,   3,   3,   3,   1,   4,   0, 475]])